In [1]:
# IMPORTING LIBRARIES
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sb
import klib as kl
import sklearn
import imblearn

In [2]:
# CONFIGURING SETTINGS
pd.set_option('display.max_columns', None)
np.set_printoptions(precision=3)
sb.set(style='darkgrid')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [8]:
# LOADING DATASETS

datasets = ['datasets/Monday-WorkingHours.pcap_ISCX.csv','datasets/Tuesday-WorkingHours.pcap_ISCX.csv',
            'datasets/Wednesday-WorkingHours.pcap_ISCX.csv','datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
            'datasets/Thursday-WorkingHours-Afternoon-Infiltration.pcap_ISCX.csv','datasets/Friday-WorkingHours-Morning.pcap_ISCX.csv',
            'datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv','datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv']

main_df = pd.DataFrame(pd.read_csv(datasets[0]))

for i in range(1,len(datasets)):
    data = pd.read_csv(datasets[i])
    df = pd.DataFrame(data)
    main_df = pd.concat([main_df,df],axis=1)
    
print(main_df)

         Destination Port   Flow Duration   Total Fwd Packets  \
0                 49188.0             4.0                 2.0   
1                 49188.0             1.0                 2.0   
2                 49188.0             1.0                 2.0   
3                 49188.0             1.0                 2.0   
4                 49486.0             3.0                 2.0   
...                   ...             ...                 ...   
692698                NaN             NaN                 NaN   
692699                NaN             NaN                 NaN   
692700                NaN             NaN                 NaN   
692701                NaN             NaN                 NaN   
692702                NaN             NaN                 NaN   

         Total Backward Packets  Total Length of Fwd Packets  \
0                           0.0                         12.0   
1                           0.0                         12.0   
2                          